
### 1. Representación Numérica de Datos  
Este fragmento de código utiliza vectores como representación numérica para manejar datos estructurados en consultas y análisis.

### 2. Bases de Datos Vectoriales  
El código implementa **ChromaDB**, una base de datos vectorial diseñada para almacenar y manejar datos de manera eficiente en formato vectorial. Esto es útil para búsquedas semánticas y clasificaciones rápidas.

### 3. Persistencia de Datos  
La configuración del directorio de persistencia (`persist_directory`) garantiza que los datos vectoriales almacenados en la base de datos sean persistentes, lo que facilita su reutilización.

### 4. Creación de Colecciones  
El ejemplo crea una colección específica (`partidos_futbol`) para organizar y consultar datos vectoriales relacionados con partidos de fútbol.


In [4]:
import os

persist_directory = r"C:/Users/Usuario/vectorialDatabaseCrud/chromadb"

# Verificar si el directorio existe
if not os.path.exists(persist_directory):
    print("El directorio no existe. Creando el directorio...")
    os.makedirs(persist_directory)
else:
    print("El directorio ya existe.")

El directorio ya existe.


In [19]:
import chromadb
from chromadb.config import Settings

# Directorio para persistir la base de datos
persist_directory = r"C:/Users/Usuario/vectorialDatabaseCrud/chromadb"

try:
    # Configuración del cliente de ChromaDB
    print("Configurando cliente de ChromaDB...")
    chroma_client = chromadb.PersistentClient(
        path=persist_directory
    )
    print("Cliente configurado exitosamente.")

    # Crear o usar una colección llamada "partidos_futbol"
    print("Intentando crear o recuperar la colección 'partidos_futbol'...")
    collection = chroma_client.get_or_create_collection(name="partidos_futbol")
    print("Colección 'partidos_futbol' creada o recuperada exitosamente.")

except Exception as e:
    print(f"Error encontrado: {e}")


Configurando cliente de ChromaDB...
Cliente configurado exitosamente.
Intentando crear o recuperar la colección 'partidos_futbol'...
Colección 'partidos_futbol' creada o recuperada exitosamente.



1. **Embeddings como Representación Numérica**  
   - Se utiliza el modelo `SentenceTransformer` para transformar textos en representaciones vectoriales numéricas (*embeddings*). Esto permite capturar relaciones semánticas entre las frases.

2. **Procesamiento de Lenguaje Natural (PLN)**  
   - El modelo de *embeddings* convierte las descripciones de partidos en vectores, esenciales para cálculos de similitud y análisis semántico.

3. **Almacenamiento Vectorial**  
   - Los datos de los partidos (incluyendo texto, metadatos y sus *embeddings*) se insertan en una colección de una base de datos vectorial para búsquedas eficientes.

4. **Relaciones entre Documentos**  
   - Los *embeddings* generados permiten medir similitudes entre los textos, facilitando consultas como "¿Qué partidos son más similares en contexto?"

5. **Metadatos y Organización Eficiente**  
   - Además del texto, se almacenan metadatos (como equipos, fecha y estadio) que enriquecen las búsquedas y análisis.

collection.add():

Este es un método que se utiliza para agregar (insertar) datos en una colección. La colección puede referirse a una base de datos o a un índice que almacena datos.
Parámetros de add():

ids=ids: Aquí se está pasando un conjunto de identificadores (ids) que serán asignados a los documentos que se insertan. Es probable que ids sea una lista o un array de identificadores únicos para cada documento.
documents=textos: Este parámetro contiene los documentos que se van a insertar. En este caso, textos parece ser una lista o un conjunto de textos (probablemente de partidos, si el contexto es un sistema de fútbol).
metadatas=metadatas: Aquí se incluyen los metadatos asociados con los documentos. Los metadatos podrían incluir información adicional sobre cada partido, como la fecha, el equipo local, el equipo visitante, entre otros.
embeddings=embeddings: Este parámetro contiene los embeddings, que son representaciones vectoriales de los textos. Los embeddings son utilizados en sistemas de búsqueda o modelos de inteligencia artificial para representar texto de manera que sea comprensible para los algoritmos, lo que permite comparar la similitud entre los documentos.
print("Partidos con embeddings insertados."):

Después de la operación de inserción, se imprime un mensaje en la consola indicando que los "partidos con embeddings" se han insertado correctamente en la colección.

In [44]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# 1. Inicializar el modelo de embeddings
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Crear o conectar a una colección en Chroma con persistencia
client = chromadb.PersistentClient(path="./chroma_db")  # Almacena la base de datos en el directorio "chroma_db"
collection = client.get_or_create_collection(name="partidos_fifa_2022")

# 3. Datos de partidos de fútbol
partidos = [
    {
        "id": "1",
        "texto": "Partido entre Qatar y Ecuador en el Al Bayt Stadium el 20 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Qatar",
            "equipo_visitante": "Ecuador",
            "fecha": "2022-11-20",
            "estadio": "Al Bayt Stadium",
            "etapa": "Fase de grupos",
            "grupo": "A",
            "resultado": "0-2"
        }
    },
    {
        "id": "2",
        "texto": "Partido entre Senegal y Países Bajos en el Al Thumama Stadium el 21 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Senegal",
            "equipo_visitante": "Países Bajos",
            "fecha": "2022-11-21",
            "estadio": "Al Thumama Stadium",
            "etapa": "Fase de grupos",
            "grupo": "A",
            "resultado": "1-2"
        }
    },
    {
        "id": "3",
        "texto": "Partido entre Qatar y Senegal en el Al Thumama Stadium el 25 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Qatar",
            "equipo_visitante": "Senegal",
            "fecha": "2022-11-25",
            "estadio": "Al Thumama Stadium",
            "etapa": "Fase de grupos",
            "grupo": "A",
            "resultado": "1-3"
        }
    },
    {
        "id": "4",
        "texto": "Partido entre Ecuador y Países Bajos en el Khalifa International Stadium el 25 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Ecuador",
            "equipo_visitante": "Países Bajos",
            "fecha": "2022-11-25",
            "estadio": "Khalifa International Stadium",
            "etapa": "Fase de grupos",
            "grupo": "A",
            "resultado": "1-1"
        }
    },
    {
        "id": "5",
        "texto": "Partido entre Inglaterra e Irán en el Khalifa International Stadium el 21 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Inglaterra",
            "equipo_visitante": "Irán",
            "fecha": "2022-11-21",
            "estadio": "Khalifa International Stadium",
            "etapa": "Fase de grupos",
            "grupo": "B",
            "resultado": "6-2"
        }
    },
    {
        "id": "6",
        "texto": "Partido entre Estados Unidos y Gales en el Al Rayyan Stadium el 21 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Estados Unidos",
            "equipo_visitante": "Gales",
            "fecha": "2022-11-21",
            "estadio": "Al Rayyan Stadium",
            "etapa": "Fase de grupos",
            "grupo": "B",
            "resultado": "1-1"
        }
    },
    {
        "id": "7",
        "texto": "Partido entre Inglaterra y Estados Unidos en el Al Bayt Stadium el 25 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Inglaterra",
            "equipo_visitante": "Estados Unidos",
            "fecha": "2022-11-25",
            "estadio": "Al Bayt Stadium",
            "etapa": "Fase de grupos",
            "grupo": "B",
            "resultado": "0-0"
        }
    },
    {
        "id": "8",
        "texto": "Partido entre Irán y Gales en el Ahmed Bin Ali Stadium el 25 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Irán",
            "equipo_visitante": "Gales",
            "fecha": "2022-11-25",
            "estadio": "Ahmed Bin Ali Stadium",
            "etapa": "Fase de grupos",
            "grupo": "B",
            "resultado": "2-0"
        }
    },
    {
        "id": "9",
        "texto": "Partido entre Argentina y Arabia Saudita en el Lusail Iconic Stadium el 22 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Argentina",
            "equipo_visitante": "Arabia Saudita",
            "fecha": "2022-11-22",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Fase de grupos",
            "grupo": "C",
            "resultado": "1-2"
        }
    },
    {
        "id": "10",
        "texto": "Partido entre México y Polonia en el Lusail Iconic Stadium el 22 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "México",
            "equipo_visitante": "Polonia",
            "fecha": "2022-11-22",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Fase de grupos",
            "grupo": "C",
            "resultado": "0-0"
        }
    },
    {
        "id": "11",
        "texto": "Partido entre Argentina y México en el Lusail Iconic Stadium el 26 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Argentina",
            "equipo_visitante": "México",
            "fecha": "2022-11-26",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Fase de grupos",
            "grupo": "C",
            "resultado": "2-0"
        }
    },
    {
        "id": "12",
        "texto": "Partido entre Polonia y Arabia Saudita en el Education City Stadium el 26 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Polonia",
            "equipo_visitante": "Arabia Saudita",
            "fecha": "2022-11-26",
            "estadio": "Education City Stadium",
            "etapa": "Fase de grupos",
            "grupo": "C",
            "resultado": "2-0"
        }
    },
    {
        "id": "13",
        "texto": "Partido entre Francia y Australia en el Al Janoub Stadium el 22 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Francia",
            "equipo_visitante": "Australia",
            "fecha": "2022-11-22",
            "estadio": "Al Janoub Stadium",
            "etapa": "Fase de grupos",
            "grupo": "D",
            "resultado": "4-1"
        }
    },
    {
        "id": "14",
        "texto": "Partido entre Dinamarca y Túnez en el Education City Stadium el 22 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Dinamarca",
            "equipo_visitante": "Túnez",
            "fecha": "2022-11-22",
            "estadio": "Education City Stadium",
            "etapa": "Fase de grupos",
            "grupo": "D",
            "resultado": "0-0"
        }
    },
    {
        "id": "15",
        "texto": "Partido entre Francia y Dinamarca en el Stadium 974 el 26 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Francia",
            "equipo_visitante": "Dinamarca",
            "fecha": "2022-11-26",
            "estadio": "Stadium 974",
            "etapa": "Fase de grupos",
            "grupo": "D",
            "resultado": "2-1"
        }
    },
    {
        "id": "16",
        "texto": "Partido entre Australia y Túnez en el Al Janoub Stadium el 26 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Australia",
            "equipo_visitante": "Túnez",
            "fecha": "2022-11-26",
            "estadio": "Al Janoub Stadium",
            "etapa": "Fase de grupos",
            "grupo": "D",
            "resultado": "1-0"
        }
    },
    {
        "id": "17",
        "texto": "Partido entre España y Costa Rica en el Al Thumama Stadium el 23 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "España",
            "equipo_visitante": "Costa Rica",
            "fecha": "2022-11-23",
            "estadio": "Al Thumama Stadium",
            "etapa": "Fase de grupos",
            "grupo": "E",
            "resultado": "7-0"
        }
    },
    {
        "id": "18",
        "texto": "Partido entre Alemania y Japón en el Khalifa International Stadium el 23 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Alemania",
            "equipo_visitante": "Japón",
            "fecha": "2022-11-23",
            "estadio": "Khalifa International Stadium",
            "etapa": "Fase de grupos",
            "grupo": "E",
            "resultado": "1-2"
        }
    },
    {
        "id": "19",
        "texto": "Partido entre España y Alemania en el Al Bayt Stadium el 27 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "España",
            "equipo_visitante": "Alemania",
            "fecha": "2022-11-27",
            "estadio": "Al Bayt Stadium",
            "etapa": "Fase de grupos",
            "grupo": "E",
            "resultado": "1-1"
        }
    },
    {
        "id": "20",
        "texto": "Partido entre Costa Rica y Japón en el Ahmad Bin Ali Stadium el 27 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Costa Rica",
            "equipo_visitante": "Japón",
            "fecha": "2022-11-27",
            "estadio": "Ahmad Bin Ali Stadium",
            "etapa": "Fase de grupos",
            "grupo": "E",
            "resultado": "0-1"
        }
    },
    {
        "id": "21",
        "texto": "Partido entre Bélgica y Canadá en el Ahmad Bin Ali Stadium el 23 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Bélgica",
            "equipo_visitante": "Canadá",
            "fecha": "2022-11-23",
            "estadio": "Ahmad Bin Ali Stadium",
            "etapa": "Fase de grupos",
            "grupo": "F",
            "resultado": "1-0"
        }
    },
    {
        "id": "22",
        "texto": "Partido entre Marruecos y Croacia en el Al Bayt Stadium el 23 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Marruecos",
            "equipo_visitante": "Croacia",
            "fecha": "2022-11-23",
            "estadio": "Al Bayt Stadium",
            "etapa": "Fase de grupos",
            "grupo": "F",
            "resultado": "0-0"
        }
    },
    {
        "id": "23",
        "texto": "Partido entre Bélgica y Marruecos en el Al Thumama Stadium el 27 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Bélgica",
            "equipo_visitante": "Marruecos",
            "fecha": "2022-11-27",
            "estadio": "Al Thumama Stadium",
            "etapa": "Fase de grupos",
            "grupo": "F",
            "resultado": "0-2"
        }
    },
    {
        "id": "24",
        "texto": "Partido entre Croacia y Canadá en el Khalifa International Stadium el 27 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Croacia",
            "equipo_visitante": "Canadá",
            "fecha": "2022-11-27",
            "estadio": "Khalifa International Stadium",
            "etapa": "Fase de grupos",
            "grupo": "F",
            "resultado": "4-1"
        }
    },
    {
        "id": "25",
        "texto": "Partido entre Suiza y Camerún en el Al Janoub Stadium el 24 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Suiza",
            "equipo_visitante": "Camerún",
            "fecha": "2022-11-24",
            "estadio": "Al Janoub Stadium",
            "etapa": "Fase de grupos",
            "grupo": "G",
            "resultado": "1-0"
        }
    },
    {
        "id": "26",
        "texto": "Partido entre Brasil y Serbia en el Lusail Iconic Stadium el 24 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Brasil",
            "equipo_visitante": "Serbia",
            "fecha": "2022-11-24",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Fase de grupos",
            "grupo": "G",
            "resultado": "2-0"
        }
    },
    {
        "id": "27",
        "texto": "Partido entre Brasil y Suiza en el Stadium 974 el 28 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Brasil",
            "equipo_visitante": "Suiza",
            "fecha": "2022-11-28",
            "estadio": "Stadium 974",
            "etapa": "Fase de grupos",
            "grupo": "G",
            "resultado": "1-0"
        }
    },
    {
        "id": "28",
        "texto": "Partido entre Serbia y Camerún en el Al Janoub Stadium el 28 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Serbia",
            "equipo_visitante": "Camerún",
            "fecha": "2022-11-28",
            "estadio": "Al Janoub Stadium",
            "etapa": "Fase de grupos",
            "grupo": "G",
            "resultado": "3-3"
        }
    },
    {
        "id": "29",
        "texto": "Partido entre Uruguay y Corea del Sur en el Education City Stadium el 24 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Uruguay",
            "equipo_visitante": "Corea del Sur",
            "fecha": "2022-11-24",
            "estadio": "Education City Stadium",
            "etapa": "Fase de grupos",
            "grupo": "H",
            "resultado": "0-0"
        }
    },
    {
        "id": "30",
        "texto": "Partido entre Portugal y Ghana en el Stadium 974 el 24 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Portugal",
            "equipo_visitante": "Ghana",
            "fecha": "2022-11-24",
            "estadio": "Stadium 974",
            "etapa": "Fase de grupos",
            "grupo": "H",
            "resultado": "3-2"
        }
    },
    {
        "id": "31",
        "texto": "Partido entre Portugal y Uruguay en el Lusail Iconic Stadium el 28 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Portugal",
            "equipo_visitante": "Uruguay",
            "fecha": "2022-11-28",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Fase de grupos",
            "grupo": "H",
            "resultado": "2-0"
        }
    },
    {
        "id": "32",
        "texto": "Partido entre Corea del Sur y Ghana en el Education City Stadium el 28 de noviembre de 2022.",
        "metadata": {
            "equipo_local": "Corea del Sur",
            "equipo_visitante": "Ghana",
            "fecha": "2022-11-28",
            "estadio": "Education City Stadium",
            "etapa": "Fase de grupos",
            "grupo": "H",
            "resultado": "2-3"
        }
    },
    {
        "id": "33",
        "texto": "Partido entre Países Bajos y Estados Unidos en el Khalifa International Stadium el 3 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Países Bajos",
            "equipo_visitante": "Estados Unidos",
            "fecha": "2022-12-03",
            "estadio": "Khalifa International Stadium",
            "etapa": "Octavos de final",
            "resultado": "3-1"
        }
    },
    {
        "id": "34",
        "texto": "Partido entre Argentina y Australia en el Ahmad Bin Ali Stadium el 3 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Argentina",
            "equipo_visitante": "Australia",
            "fecha": "2022-12-03",
            "estadio": "Ahmad Bin Ali Stadium",
            "etapa": "Octavos de final",
            "resultado": "2-1"
        }
    },
    {
        "id": "35",
        "texto": "Partido entre Francia y Polonia en el Al Thumama Stadium el 4 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Francia",
            "equipo_visitante": "Polonia",
            "fecha": "2022-12-04",
            "estadio": "Al Thumama Stadium",
            "etapa": "Octavos de final",
            "resultado": "3-1"
        }
    },
    {
        "id": "36",
        "texto": "Partido entre Inglaterra y Senegal en el Al Bayt Stadium el 4 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Inglaterra",
            "equipo_visitante": "Senegal",
            "fecha": "2022-12-04",
            "estadio": "Al Bayt Stadium",
            "etapa": "Octavos de final",
            "resultado": "3-0"
        }
    },
    {
        "id": "37",
        "texto": "Partido entre Japón y Croacia en el Al Janoub Stadium el 5 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Japón",
            "equipo_visitante": "Croacia",
            "fecha": "2022-12-05",
            "estadio": "Al Janoub Stadium",
            "etapa": "Octavos de final",
            "resultado": "1-1 (1-3 penales)"
        }
    },
    {
        "id": "38",
        "texto": "Partido entre Brasil y Corea del Sur en el Stadium 974 el 5 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Brasil",
            "equipo_visitante": "Corea del Sur",
            "fecha": "2022-12-05",
            "estadio": "Stadium 974",
            "etapa": "Octavos de final",
            "resultado": "4-1"
        }
    },
    {
        "id": "39",
        "texto": "Partido entre Marruecos y España en el Education City Stadium el 6 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Marruecos",
            "equipo_visitante": "España",
            "fecha": "2022-12-06",
            "estadio": "Education City Stadium",
            "etapa": "Octavos de final",
            "resultado": "0-0 (3-0 penales)"
        }
    },
    {
        "id": "40",
        "texto": "Partido entre Portugal y Suiza en el Lusail Iconic Stadium el 6 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Portugal",
            "equipo_visitante": "Suiza",
            "fecha": "2022-12-06",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Octavos de final",
            "resultado": "6-1"
        }
    },
    {
        "id": "41",
        "texto": "Partido entre Croacia y Brasil en el Education City Stadium el 9 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Croacia",
            "equipo_visitante": "Brasil",
            "fecha": "2022-12-09",
            "estadio": "Education City Stadium",
            "etapa": "Cuartos de final",
            "resultado": "1-1 (4-2 penales)"
        }
    },
    {
        "id": "42",
        "texto": "Partido entre Países Bajos y Argentina en el Lusail Iconic Stadium el 9 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Países Bajos",
            "equipo_visitante": "Argentina",
            "fecha": "2022-12-09",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Cuartos de final",
            "resultado": "2-2 (3-4 penales)"
        }
    },
    {
        "id": "43",
        "texto": "Partido entre Marruecos y Portugal en el Al Thumama Stadium el 10 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Marruecos",
            "equipo_visitante": "Portugal",
            "fecha": "2022-12-10",
            "estadio": "Al Thumama Stadium",
            "etapa": "Cuartos de final",
            "resultado": "1-0"
        }
    },
    {
        "id": "44",
        "texto": "Partido entre Inglaterra y Francia en el Al Bayt Stadium el 10 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Inglaterra",
            "equipo_visitante": "Francia",
            "fecha": "2022-12-10",
            "estadio": "Al Bayt Stadium",
            "etapa": "Cuartos de final",
            "resultado": "1-2"
        }
    },
    {
        "id": "45",
        "texto": "Partido entre Argentina y Croacia en el Lusail Iconic Stadium el 13 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Argentina",
            "equipo_visitante": "Croacia",
            "fecha": "2022-12-13",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Semifinal",
            "resultado": "3-0"
        }
    },
    {
        "id": "46",
        "texto": "Partido entre Francia y Marruecos en el Al Bayt Stadium el 14 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Francia",
            "equipo_visitante": "Marruecos",
            "fecha": "2022-12-14",
            "estadio": "Al Bayt Stadium",
            "etapa": "Semifinal",
            "resultado": "2-0"
        }
    },
    {
        "id": "47",
        "texto": "Partido entre Croacia y Marruecos en el Khalifa International Stadium el 17 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Croacia",
            "equipo_visitante": "Marruecos",
            "fecha": "2022-12-17",
            "estadio": "Khalifa International Stadium",
            "etapa": "Tercer puesto",
            "resultado": "2-1"
        }
    },
    {
        "id": "48",
        "texto": "Partido entre Argentina y Francia en el Lusail Iconic Stadium el 18 de diciembre de 2022.",
        "metadata": {
            "equipo_local": "Argentina",
            "equipo_visitante": "Francia",
            "fecha": "2022-12-18",
            "estadio": "Lusail Iconic Stadium",
            "etapa": "Final",
            "resultado": "3-3 (4-2 penales)"
        }
    }
]

# 4. Preparar los datos para insertar en ChromaDB
textos = [partido["texto"] for partido in partidos]
ids = [partido["id"] for partido in partidos]
metadatas = [partido["metadata"] for partido in partidos]
embeddings = modelo_embeddings.encode(textos).tolist()

# 5. Insertar partidos con embeddings en la colección
collection.add(
    ids=ids,
    documents=textos,
    metadatas=metadatas,
    embeddings=embeddings
)

# 6. Verificar que la colección se creó correctamente
print("Colecciones existentes:", client.list_collections())
print("Partidos con embeddings insertados correctamente.")

Colecciones existentes: ['partidos_fifa_2022']
Partidos con embeddings insertados correctamente.


comparación de embeddings, específicamente en esta parte:
# Generar embedding para la consulta
consulta = "Partido entre Barcelona y Real Madrid en el Camp Nou"
embedding_consulta = modelo_embeddings.encode([consulta]).tolist()

# Buscar los documentos más similares
resultados_similares = collection.query(query_embeddings=embedding_consulta, n_results=2)

Esta sección realiza una búsqueda por similitud semántica donde:

Primero se genera un embedding para la consulta usando el modelo 'all-MiniLM-L6-v2'
Luego, ChromaDB compara internamente este embedding con los embeddings de todos los documentos en la colección
El método query() devuelve los 2 documentos más similares (n_results=2)

La comparación se realiza utilizando la similitud coseno entre los vectores de embeddings, aunque esto sucede internamente en ChromaDB. La similitud coseno mide el ángulo entre dos vectores, donde:

-Un valor más cercano a 1 indica mayor similitud
-Un valor más cercano a 0 indica menor similitud

In [84]:
# Generar embedding para la consulta
consulta = "Croacia y Argentina"
embedding_consulta = modelo_embeddings.encode([consulta]).tolist()

# Buscar los documentos más similares
resultados_similares = collection.query(query_embeddings=embedding_consulta, n_results=2)
print("Partidos similares:", resultados_similares)


Partidos similares: {'ids': [['45', '22']], 'embeddings': None, 'documents': [['Partido entre Argentina y Croacia en el Lusail Iconic Stadium el 13 de diciembre de 2022.', 'Partido entre Marruecos y Croacia en el Al Bayt Stadium el 23 de noviembre de 2022.']], 'uris': None, 'data': None, 'metadatas': [[{'equipo_local': 'Argentina', 'equipo_visitante': 'Croacia', 'estadio': 'Lusail Iconic Stadium', 'etapa': 'Semifinal', 'fecha': '2022-12-13', 'resultado': '3-0'}, {'equipo_local': 'Marruecos', 'equipo_visitante': 'Croacia', 'estadio': 'Al Bayt Stadium', 'etapa': 'Fase de grupos', 'fecha': '2022-11-23', 'grupo': 'F', 'resultado': '0-0'}]], 'distances': [[0.723682953491408, 0.841496408652508]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


### Búsqueda y Filtro por Texto Exacto

Este código realiza una búsqueda semántica en la colección utilizando una consulta de texto y aplica un filtro para obtener coincidencias exactas.

1. **Consulta Inicial**:
   - Se busca `"Camp Nou"` en la colección, obteniendo documentos, metadatos y distancias.
   - El número de resultados se limita a `n_results=2`.

2. **Filtro de Coincidencias Exactas**:
   - Se filtran resultados cuyo texto contiene `"Camp Nou"` o cuyo metadato `estadio` coincide exactamente con `"Camp Nou"`.

3. **Impresión de Resultados**:
   - Los documentos filtrados se muestran con su texto, metadatos y distancia.

In [72]:
def consultar_partidos_flexible(query_texto, n_resultados=10, filtros=None):
    """
    Realiza una consulta semántica y aplica filtros opcionales sobre los metadatos.

    Parámetros:
        query_texto (str): Texto de la consulta (pregunta en lenguaje natural).
        n_resultados (int): Número de resultados a devolver.
        filtros (dict): Filtros opcionales para aplicar sobre los metadatos.
                        Ejemplo: {"equipo_local": "Brasil", "etapa": "Cuartos de final"}
    """
    # Generar embedding para la consulta
    query_embedding = modelo_embeddings.encode(query_texto).tolist()

    # Realizar la consulta en Chroma
    resultados = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_resultados,
        include=["documents", "metadatas", "distances"]
    )

    # Formatear resultados
    resultados_formateados = [
        {
            "texto": doc,
            "metadata": meta,
            "distancia": dist
        }
        for doc, meta, dist in zip(
            resultados["documents"][0],
            resultados["metadatas"][0],
            resultados["distances"][0]
        )
    ]

    # Aplicar filtros si se proporcionan
    if filtros:
        resultados_filtrados = []
        for res in resultados_formateados:
            cumple_filtros = True
            for clave, valor in filtros.items():
                if clave not in res["metadata"] or res["metadata"][clave] != valor:
                    cumple_filtros = False
                    break
            if cumple_filtros:
                resultados_filtrados.append(res)
        return resultados_filtrados

    return resultados_formateados

# Ejemplo de consulta flexible
query_texto = "2-0"
filtros = {"etapa": "Cuartos de final"}  # Filtro opcional
resultados = consultar_partidos_flexible(query_texto, n_resultados=10, filtros=filtros)

# Mostrar resultados
if resultados:
    for i, res in enumerate(resultados, start=1):
        print(f"\nResultado {i}:")
        print(f"Texto: {res['texto']}")
        print(f"Metadata: {res['metadata']}")
        print(f"Distancia (similitud): {res['distancia']:.4f}")
else:
    print("No se encontraron resultados que coincidan con la consulta y los filtros.")


Resultado 1:
Texto: Partido entre Países Bajos y Argentina en el Lusail Iconic Stadium el 9 de diciembre de 2022.
Metadata: {'equipo_local': 'Países Bajos', 'equipo_visitante': 'Argentina', 'estadio': 'Lusail Iconic Stadium', 'etapa': 'Cuartos de final', 'fecha': '2022-12-09', 'resultado': '2-2 (3-4 penales)'}
Distancia (similitud): 1.5567


### Búsqueda Filtrada por Metadatos

Este código realiza una búsqueda semántica con un filtro adicional aplicado sobre los metadatos.

- **Consulta**: Busca documentos relacionados con `"Barcelona"`.
- **Filtro**: Incluye solo resultados donde `equipo_local` es `"Barcelona"`.
- **Resultados**: Imprime los documentos, metadatos, y distancias relevantes.

In [81]:
def consultar_por_resultado(resultado, n_resultados=5):
    """
    Realiza una consulta para encontrar partidos con un resultado específico.

    Parámetros:
        resultado (str): El resultado a buscar (por ejemplo, "1-0", "3-2").
        n_resultados (int): Número de resultados a devolver.
    """
    # Realizar la consulta en Chroma sin filtros iniciales
    resultados = collection.get(
        include=["documents", "metadatas"]
    )

    # Filtrar manualmente los resultados que coincidan con el resultado buscado
    resultados_filtrados = [
        {
            "texto": doc,
            "metadata": meta
        }
        for doc, meta in zip(resultados["documents"], resultados["metadatas"])
        if meta.get("resultado") == resultado
    ]

    # Limitar el número de resultados
    return resultados_filtrados[:n_resultados]

# 4. Ejemplo de consulta por resultado
resultado_buscado = "Francia y Argentina"  # Puedes cambiar esto a cualquier resultado
resultados = consultar_por_resultado(resultado_buscado, n_resultados=3)

# 5. Mostrar resultados
print(f"Partidos con resultado '{resultado_buscado}':")
for i, res in enumerate(resultados, start=1):
    print(f"\nPartido {i}:")
    print(f"Texto: {res['texto']}")
    print(f"Metadata: {res['metadata']}")

Partidos con resultado 'Francia y Argentina':


### Actualización de Partido y su Embedding

Este código actualiza la información de un partido en la colección, incluyendo su texto, metadatos, y embedding.

1. **Generación del Nuevo Embedding**:
   - Se crea un nuevo embedding para el texto actualizado del partido.

2. **Eliminación del Partido Antiguo**:
   - Se elimina el partido existente utilizando su ID (`"1"`).

3. **Adición del Partido Actualizado**:
   - Se agrega el partido actualizado con:
     - ID: `"1"`
     - Texto: Descripción del partido.
     - Metadatos: Incluyen detalles como equipos, fecha y estadio.
     - Embedding: Generado previamente.

4. **Confirmación**:
   - Imprime un mensaje indicando que el partido ha sido actualizado.

In [33]:
# Actualizar un partido y su embedding
nuevo_texto = "Partido entre Barcelona y Real Madrid en el Camp Nou el 25 de diciembre de 2025, con 70,000 espectadores."
nuevo_embedding = modelo_embeddings.encode([nuevo_texto]).tolist()
collection.delete(ids=["1"])  # Eliminar partido antiguo
collection.add(ids=["1"], documents=[nuevo_texto], metadatas=[{"equipo_local": "Barcelona", "equipo_visitante": "Real Madrid", "fecha": "2025-12-25", "estadio": "Camp Nou"}], embeddings=nuevo_embedding)
print("Partido actualizado con nuevo embedding.")

Delete of nonexisting embedding ID: 1
Delete of nonexisting embedding ID: 1


Partido actualizado con nuevo embedding.


Búsqueda por Equipo Local

Este código realiza una búsqueda en la colección filtrando por un equipo local específico, en este caso, "Barcelona".

1. **Consulta Vacía**:
   - Se realiza una búsqueda sin texto de consulta (`query_texts=[""]`), lo que indica que la búsqueda se centrará únicamente en los metadatos.

2. **Filtro por Metadata**:
   - Se usa el filtro `where={"equipo_local": "Barcelona"}` para obtener solo los documentos donde el equipo local sea "Barcelona".

3. **Resultados**:
   - Se obtienen hasta 5 resultados (`n_results=5`), y se incluyen los documentos y metadatos de los partidos filtrados.

4. **Impresión de Resultados**:
   - Se muestran los partidos encontrados, incluyendo el texto del partido y los metadatos relacionados.

Partidos jugados en el estadio 'Al Bayt Stadium':

Partido 1:
Texto: Partido entre Qatar y Ecuador en el Al Bayt Stadium el 20 de noviembre de 2022.
Metadata: {'equipo_local': 'Qatar', 'equipo_visitante': 'Ecuador', 'estadio': 'Al Bayt Stadium', 'etapa': 'Fase de grupos', 'fecha': '2022-11-20', 'grupo': 'A', 'resultado': '0-2'}

Partido 2:
Texto: Partido entre Inglaterra y Estados Unidos en el Al Bayt Stadium el 25 de noviembre de 2022.
Metadata: {'equipo_local': 'Inglaterra', 'equipo_visitante': 'Estados Unidos', 'estadio': 'Al Bayt Stadium', 'etapa': 'Fase de grupos', 'fecha': '2022-11-25', 'grupo': 'B', 'resultado': '0-0'}

Partido 3:
Texto: Partido entre España y Alemania en el Al Bayt Stadium el 27 de noviembre de 2022.
Metadata: {'equipo_local': 'España', 'equipo_visitante': 'Alemania', 'estadio': 'Al Bayt Stadium', 'etapa': 'Fase de grupos', 'fecha': '2022-11-27', 'grupo': 'E', 'resultado': '1-1'}


### Búsqueda por Estadio

Este código realiza una búsqueda en la colección filtrando por un estadio específico, en este caso, "Camp Nou".

1. **Consulta Vacía**:
   - Se realiza una búsqueda sin texto de consulta (`query_texts=[""]`), centrando la búsqueda exclusivamente en los metadatos.

2. **Filtro por Metadata (Estadio)**:
   - Se utiliza el filtro `where={"estadio": "Camp Nou"}` para buscar documentos relacionados con partidos jugados en el "Camp Nou".

3. **Resultados**:
   - Se limitan a los primeros 5 resultados (`n_results=5`), y se incluyen tanto los documentos como los metadatos.

4. **Impresión de Resultados**:
   - Se muestran los partidos encontrados, incluyendo el texto del partido y los metadatos asociados.

Ejemplo de uso:

In [86]:

def consultar_por_estadio(nombre_estadio, n_resultados=5):
    """
    Realiza una consulta para encontrar partidos jugados en un estadio específico.

    Parámetros:
        nombre_estadio (str): Nombre del estadio a buscar (por ejemplo, "Al Bayt Stadium").
        n_resultados (int): Número de resultados a devolver.
    """
    # Realizar la consulta en Chroma sin filtros iniciales
    resultados = collection.get(
        include=["documents", "metadatas"]
    )

    # Filtrar manualmente los resultados que coincidan con el estadio buscado
    resultados_filtrados = [
        {
            "texto": doc,
            "metadata": meta
        }
        for doc, meta in zip(resultados["documents"], resultados["metadatas"])
        if meta.get("estadio") == nombre_estadio
    ]

    # Limitar el número de resultados
    return resultados_filtrados[:n_resultados]

# 4. Ejemplo de consulta por estadio
estadio_buscado = "Lusail Iconic Stadium"  # Puedes cambiar esto a cualquier estadio
resultados = consultar_por_estadio(estadio_buscado, n_resultados=3)

# 5. Mostrar resultados
print(f"Partidos jugados en el estadio '{estadio_buscado}':")
for i, res in enumerate(resultados, start=1):
    print(f"\nPartido {i}:")
    print(f"Texto: {res['texto']}")
    print(f"Metadata: {res['metadata']}")

Partidos jugados en el estadio 'Lusail Iconic Stadium':

Partido 1:
Texto: Partido entre Argentina y Arabia Saudita en el Lusail Iconic Stadium el 22 de noviembre de 2022.
Metadata: {'equipo_local': 'Argentina', 'equipo_visitante': 'Arabia Saudita', 'estadio': 'Lusail Iconic Stadium', 'etapa': 'Fase de grupos', 'fecha': '2022-11-22', 'grupo': 'C', 'resultado': '1-2'}

Partido 2:
Texto: Partido entre México y Polonia en el Lusail Iconic Stadium el 22 de noviembre de 2022.
Metadata: {'equipo_local': 'México', 'equipo_visitante': 'Polonia', 'estadio': 'Lusail Iconic Stadium', 'etapa': 'Fase de grupos', 'fecha': '2022-11-22', 'grupo': 'C', 'resultado': '0-0'}

Partido 3:
Texto: Partido entre Argentina y México en el Lusail Iconic Stadium el 26 de noviembre de 2022.
Metadata: {'equipo_local': 'Argentina', 'equipo_visitante': 'México', 'estadio': 'Lusail Iconic Stadium', 'etapa': 'Fase de grupos', 'fecha': '2022-11-26', 'grupo': 'C', 'resultado': '2-0'}


### Búsqueda por Fecha Específica

Este código realiza una búsqueda en la colección filtrando por una fecha específica.

1. **Consulta Vacía**:
   - Se realiza una búsqueda sin texto de consulta (`query_texts=[""]`), lo que indica que la búsqueda se basa únicamente en los metadatos.

2. **Filtro por Fecha**:
   - Se utiliza el filtro `where={"fecha": "2025-12-25"}` para buscar documentos relacionados con partidos ocurridos en la fecha "2025-12-25".

3. **Resultados**:
   - Se limitan a los primeros 5 resultados (`n_results=5`), incluyendo tanto los documentos como los metadatos de los partidos.

4. **Impresión de Resultados**:
   - Se muestran los partidos encontrados, mostrando el texto del partido y los metadatos asociados.

In [90]:
# Búsqueda por fecha específica
resultados_fecha = collection.query(
    query_texts=[""],
    n_results=5,
    where={"fecha": "2022-12-10"},
    include=["documents", "metadatas"]
)

for i, (doc, meta) in enumerate(zip(resultados_fecha['documents'][0], resultados_fecha['metadatas'][0])):
    print(f"\nPartido {i+1}:")
    print(f"Texto: {doc}")
    print(f"Metadata: {meta}")


Partido 1:
Texto: Partido entre Inglaterra y Francia en el Al Bayt Stadium el 10 de diciembre de 2022.
Metadata: {'equipo_local': 'Inglaterra', 'equipo_visitante': 'Francia', 'estadio': 'Al Bayt Stadium', 'etapa': 'Cuartos de final', 'fecha': '2022-12-10', 'resultado': '1-2'}

Partido 2:
Texto: Partido entre Marruecos y Portugal en el Al Thumama Stadium el 10 de diciembre de 2022.
Metadata: {'equipo_local': 'Marruecos', 'equipo_visitante': 'Portugal', 'estadio': 'Al Thumama Stadium', 'etapa': 'Cuartos de final', 'fecha': '2022-12-10', 'resultado': '1-0'}


Este código actualiza los metadatos de un partido específico en la colección.

1. **Actualización de Metadata**:
   - Se actualizan los metadatos del partido con ID `"2"`, incluyendo información como:
     - **Equipo Local**: Bucaramanga
     - **Equipo Visitante**: Liverpool
     - **Fecha**: 2025-01-10
     - **Estadio**: Old Trafford
     - **Estado del Partido**: Finalizado
     - **Resultado**: 2-1

2. **Confirmación**:
   - Imprime un mensaje confirmando que los metadatos se han actualizado correctamente.


In [37]:
collection.update(
    ids=["2"],
    metadatas=[{
        "equipo_local": "Bucaramanga",
        "equipo_visitante": "Liverpool",
        "fecha": "2025-01-10",
        "estadio": "Old Trafford",
        "estado_partido": "Finalizado",
        "resultado": "2-1"
    }]
)

print("Metadata actualizada usando update")

Metadata actualizada usando update


### Eliminación de Documento por ID

Este código elimina un documento específico de la colección utilizando su ID.

1. **Eliminación del Documento**:
   - Se elimina el documento con el ID `"9"` de la colección.

2. **Confirmación**:
   - Imprime un mensaje indicando que el documento con ID `"9"` ha sido eliminado correctamente.


In [43]:
collection.delete(ids=["48"])
print("Documento con ID 48 eliminado.")

Documento con ID 48 eliminado.
